# 확률 네트워크기반 친환경 자동차 충전소(GVCS) 로케이션 인텔리전스(Location Intelligence) 대시보드

## 제안 배경

**친환경 자동차 생산에만 관심이 쏠려 사용자를 고려하지 못한 설치로 인한 잉여 충전소 발생**

- 민관 협력을 통해 충전 서비스 관련 사업이 꾸준히 증가함. 하지만 전국에 설치된 공공 급속충전기 총 2,896개(2021년 기준) 중 하루 1회 이하로 사용한 충전기가 1,164개로 전체의 40.1%를 차지하고 있다. 심지어 17개는 전혀 사용하지 않은 것으로 나타남.

- 이와 같은 충전소마다 이용 편차가 큰 이유는 전기차 보급 목표에 따라 구역별로  충전소를 나눠 설치했기 때문이라는 지적도 있다.

- 한국전기차충전협동조합은 앞으로 수요가 몰리는 거점을 중심을 중심으로 충전기 설치를 해야하 한다는 의견 주장함.

- 기존의 정부 중심으로 공급자 입장에서 설치하던 충전소를 친환경 자동차 차주를 고려해 이용자 중심으로 변화 설치가 필요하다는 지적도 있다.

즉, **꾸준히 증가하는 친환경 차량의 수요를 위해 저비용 고효율의 충전소 설치를 목표 한다.** 충전소 설치 사업자의 기대 수익 최적화를 위해 사용자 입장을 고려한 체계적이며 최적의 입지 선정 결과를 제시함.

## 사용 데이터

1. 충전소 위치 및 현황 정보
2. 급속충전기 보급 현황
3. LPG충전소 현황
4. 오존 월별 대기오염도
5. 아황산가스 월별 대기오염도
6. 이산화질소 월별 대기오염도
7. 미세먼지 월별 대기오염도
8. 행정구역별 인구수
9. 항만 현황
10. 지역별 전기차 등록대 수

In [405]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
import networkx as nx
import numpy as np
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go

## 상위 요소 - 환경적 요인

대기오염도를 기반하여 확률 값을 계산한다.

### 오존 월별 대기오염도 데이터 불러오기

In [406]:
ozone_file_path = "C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\오존_월별_도시별_대기오염도.csv"
df_ozone = pd.read_csv(ozone_file_path, encoding='cp949')
df_ozone

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.029,0.034,0.043,0.042,0.044,0.033
1,서울특별시,서울특별시,0.022,0.025,0.037,0.036,0.041,0.038
2,부산광역시,부산광역시,0.031,0.036,0.043,0.041,0.041,0.028
3,대구광역시,대구광역시,0.025,0.032,0.043,0.040,0.045,0.031
4,인천광역시,인천광역시,0.026,0.030,0.040,0.042,0.044,0.041
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.034,0.039,0.050,0.048,0.051,0.038**
168,경상남도,도평균,0.032,0.037,0.044,0.044,0.045,0.031
169,제주특별자치도,제주특별자치도,0.044,0.046,0.051,0.051,0.045,0.029
170,제주특별자치도,서귀포,0.045,0.048,0.054,0.052,0.046,0.028


In [407]:
df_ozone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구분(1)    172 non-null    object
 1   구분(2)    172 non-null    object
 2   2021.02  172 non-null    object
 3   2021.03  172 non-null    object
 4   2021.04  172 non-null    object
 5   2021.05  172 non-null    object
 6   2021.06  172 non-null    object
 7   2021.07  172 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


In [408]:
ozen_col =  df_ozone.iloc[:,2:-1].columns.tolist()

In [409]:
df_ozone.drop(columns=ozen_col, inplace=True)

In [410]:
def advanced_replace(df, col, str, regex):
    '''
    DataFrame 특정 열의 특수문자 제거
    :param df: 대상 DataFrame
    :param col: column
    :param str: 제거할 특수 문자.
    :param regex: 적용할 정규표현식
    :return:
    '''
    df[col] = df[col].replace(str,0)
    df[col] = df[col].replace(to_replace=regex, value=r'', regex=True)
    return df

In [411]:
# 해당 컬럼 float 타입으로 변경
advanced_replace(df_ozone, df_ozone.iloc[:,2:].columns.tolist(), '-', r'[^0-9.0-9]')
df_ozone['2021.07'] = df_ozone['2021.07'].astype(float)
df_ozone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [412]:
def show_norm(mean,std,min,max):
     '''
     정규분포 함수를 matplotlib을 통해 시각화
     mean: 평균값
     std: 표준편차
     min: 관측된 데이터의 최솟값
     max: 관측된 데이터의 최댓값
     '''
     min = min - std
     # x값 설정(min에서 max까지 unit만큼씩 증가하는 x)
     x = np.linspace(min,max,100)
     # y값은 평균이 mean이고 표준편차가 std인 확률밀도함수
     fig = px.line(x=x,y=norm.pdf(x,loc=mean,scale=std))
     fig.show()

In [413]:
mean = df_ozone.iloc[:,2].mean()
std = df_ozone.iloc[:,2].std()
min = df_ozone.iloc[:,2].min()
max = df_ozone.iloc[:,2].max()
show_norm(mean,std,min,max)

### 부산광역시 오존의 누적확률 구하기

In [414]:
def cal_norm(mean,std,min,max,value):
     '''
     정규분포에서 value에 대한 누적확률 구하기
     mean: 평균값
     std: 표준편차
     min: 관측된 데이터의 최솟값
     max: 관측된 데이터의 최댓값
     value: 누적확률을 구하고자 하는 데이터의 측정값
     '''
     min = min - std
     red ='rgb(239,85,59)'
     blue = 'rgb(100,110,250)'
     fig = go.Figure()
     
     # 확률 값을 구할 특정 구간의 범위 설정
     cum_a = np.linspace(min,value,100)
     cum_b = np.linspace(value,max,100)
     
     pro = norm(mean,std).cdf(value)-norm(mean,std).cdf(min).round(3)
     # 구간 사이에 색을 채움
     fig.add_trace(go.Scatter(x=cum_a,y=norm.pdf(cum_a,mean,std), fill = 'tozeroy',name='적합',text=pro,line=dict(color=blue)))
     fig.add_trace(go.Scatter(x=cum_b,y=norm.pdf(cum_b,mean,std), fill = 'tozeroy',name='부적합',text=pro,line=dict(color=red)))
     annotations =[]
     annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(pro,3), font=dict(size=15,color=blue), xshift=-40, yshift=-100, bordercolor=blue,borderwidth=2))
     annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(1-pro,3),font=dict(size=15,color=red),xshift=40, yshift=-100, bordercolor=red,borderwidth=2))
     fig.update_layout(annotations=annotations)
     # value까지의 누적확률에서 min까지의 누적확률을 뺌
     fig.show()

     # 최종 누적확률 반환
     return pro

In [415]:
busan_oz = df_ozone[df_ozone['구분(2)']=='부산광역시'].loc[2,'2021.07']
ozone_pro = cal_norm(mean,std,min,max,busan_oz)
ozone_pro

0.23346014669602194

### 아황산가스 월별 데이터 불러오기

In [416]:
so2_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\아황산가스_월별_도시별_대기오염도.csv'
df_so2 = pd.read_csv(so2_file_path, encoding='cp949')
df_so2

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.003,0.003,0.003,0.003,0.003,0.003
1,서울특별시,서울특별시,0.003,0.003,0.003,0.003,0.003,0.003
2,부산광역시,부산광역시,0.004,0.003,0.003,0.002,0.003,0.003
3,대구광역시,대구광역시,0.002,0.002,0.002,0.002,0.002,0.002
4,인천광역시,인천광역시,0.004,0.003,0.003,0.003,0.003,0.003
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.003,0.003,0.003,0.003,0.003,0.003**
168,경상남도,도평균,0.003,0.003,0.003,0.003,0.003,0.003
169,제주특별자치도,제주특별자치도,0.002,0.001,0.002,0.002,0.002,0.002
170,제주특별자치도,서귀포,0.001,0.001,0.001,0.001,0.001,0.001


In [417]:
df_so2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구분(1)    172 non-null    object
 1   구분(2)    172 non-null    object
 2   2021.02  172 non-null    object
 3   2021.03  172 non-null    object
 4   2021.04  172 non-null    object
 5   2021.05  172 non-null    object
 6   2021.06  172 non-null    object
 7   2021.07  172 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


In [418]:
so2_col = df_so2.iloc[:,2:-1].columns.tolist()

In [419]:
df_so2.drop(columns=so2_col, inplace=True)
df_so2

,구분(1),구분(2),2021.07
0,총계,소계,0.003
1,서울특별시,서울특별시,0.003
2,부산광역시,부산광역시,0.003
3,대구광역시,대구광역시,0.002
4,인천광역시,인천광역시,0.003
...,...,...,...
167,경상남도,합천,0.003**
168,경상남도,도평균,0.003
169,제주특별자치도,제주특별자치도,0.002
170,제주특별자치도,서귀포,0.001


In [420]:
advanced_replace(df_so2,'2021.07','-',r'[^0-9.0-9]')
df_so2['2021.07'] = df_so2['2021.07'].astype(float)

In [421]:
df_so2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [422]:
df_so2.describe()

,2021.07
count,172.000000
mean,0.002674
std,0.000764
min,0.001000
25%,0.002000
50%,0.003000
75%,0.003000
max,0.005000


In [423]:
mean = df_so2.iloc[:,2].mean()
std = df_so2.iloc[:,2].std()
min = df_so2.iloc[:,2].min()
max = df_so2.iloc[:,2].max()
show_norm(mean,std,min,max)

### 부산광역시 아황산가스의 누적확률 구하기

In [424]:
busan_so2 = df_so2[df_so2['구분(2)']=='부산광역시'].loc[2,'2021.07']
so2_pro = cal_norm(mean,std,min,max,busan_so2)
so2_pro

0.6640367362231872

### 이산화질소 월별 데이터 불러오기

In [425]:
no2_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\이산화질소_월별_도시별_대기오염도.csv'
df_no2 = pd.read_csv(no2_file_path, encoding='cp949')
df_no2

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.019,0.019,0.014,0.012,0.011,0.010
1,서울특별시,서울특별시,0.030,0.032,0.022,0.019,0.018,0.016
2,부산광역시,부산광역시,0.018,0.017,0.015,0.014,0.014,0.010
3,대구광역시,대구광역시,0.022,0.018,0.013,0.012,0.010,0.009
4,인천광역시,인천광역시,0.024,0.028,0.021,0.017,0.017,0.014
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.014,0.011,0.010,0.009,0.010,0.009**
168,경상남도,도평균,0.016,0.014,0.013,0.011,0.012,0.009
169,제주특별자치도,제주특별자치도,0.011,0.011,0.010,0.010,0.009,0.006
170,제주특별자치도,서귀포,0.007,0.007,0.006,0.006,0.005,0.004


In [426]:
no2_col = df_no2.iloc[:,2:-1].columns.tolist()

In [427]:
df_no2.drop(columns=no2_col, inplace=True)

In [428]:
advanced_replace(df_no2,'2021.07','-',r'[^0-9.0-9]')
df_no2['2021.07'] = df_no2['2021.07'].astype(float)

In [429]:
df_no2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [430]:
df_no2.describe()

,2021.07
count,172.000000
mean,0.007610
std,0.003294
min,0.003000
25%,0.005000
50%,0.007000
75%,0.009250
max,0.019000


In [431]:
mean = df_no2.iloc[:,2].mean()
std = df_no2.iloc[:,2].std()
min = df_no2.iloc[:,2].min()
max = df_no2.iloc[:,2].max()
show_norm(mean,std,min,max)

### 부산광역시 이산화질소 누적확률 구하기

In [432]:
busan_no2 = df_no2[df_no2['구분(2)']=='부산광역시'].loc[2,'2021.07']
no2_pro = cal_norm(mean,std,min,max,busan_no2)

### 일산화탄소 월별 데이터 불러오기

In [433]:
co_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\일산화탄소_월별_도시별_대기오염도.csv'
df_co = pd.read_csv(co_file_path, encoding='cp949')
df_co

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.5,0.5,0.4,0.4,0.4,0.3
1,서울특별시,서울특별시,0.6,0.5,0.4,0.4,0.4,0.3
2,부산광역시,부산광역시,0.4,0.4,0.3,0.3,0.3,0.2
3,대구광역시,대구광역시,0.5,0.4,0.3,0.4,0.4,0.3
4,인천광역시,인천광역시,0.6,0.6,0.4,0.4,0.4,0.4
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.4,0.3,0.3,0.3,0.3,0.2**
168,경상남도,도평균,0.5,0.4,0.4,0.4,0.4,0.3
169,제주특별자치도,제주특별자치도,0.3,0.3,0.3,0.3,0.3,0.2
170,제주특별자치도,서귀포,0.3,0.3,0.3,0.3,0.2,0.2


In [434]:
co_col = df_co.iloc[:,2:-1].columns.tolist()

In [435]:
df_co.drop(columns=co_col, inplace=True)

In [436]:
advanced_replace(df_co, '2021.07','-',r'[^0-9.0-9]')
df_co['2021.07'] = df_co['2021.07'].astype(float)

In [437]:
df_co.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [438]:
df_co.describe()

,2021.07
count,172.000000
mean,0.328488
std,0.084825
min,0.100000
25%,0.300000
50%,0.300000
75%,0.400000
max,0.600000


In [439]:
mean = df_co.iloc[:,2].mean()
std = df_co.iloc[:,2].std()
min = df_co.iloc[:,2].min()
max = df_co.iloc[:,2].max()
show_norm(mean,std,min,max)

### 부산광역시 일산화탄소 누적확률 구하기

In [440]:
busan_co = df_co[df_co['구분(2)']=='부산광역시'].loc[2,'2021.07']
co_pro = cal_norm(mean,std,min,max,busan_co)

### 미세먼지 월별 데이터 불러오기

In [441]:
pm25_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\미세먼지_PM2.5__월별_도시별_대기오염도.csv'
df_pm25 = pd.read_csv(pm25_file_path, encoding='cp949')
df_pm25

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,25,27,17,19,19,12
1,서울특별시,서울특별시,29,32,19,20,19,15
2,부산광역시,부산광역시,21,21,15,16,15,9
3,대구광역시,대구광역시,24,23,17,18,20,10
4,인천광역시,인천광역시,28,33,19,20,23,19
...,...,...,...,...,...,...,...,...
167,경상남도,합천,18,17,14,15,18,11**
168,경상남도,도평균,19,19,15,15,16,12*
169,제주특별자치도,제주특별자치도,18,24,14,16,14,8
170,제주특별자치도,서귀포,16,23,14,16,12,8


In [442]:
df_pm25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구분(1)    172 non-null    object
 1   구분(2)    172 non-null    object
 2   2021.02  172 non-null    object
 3   2021.03  172 non-null    object
 4   2021.04  172 non-null    object
 5   2021.05  172 non-null    object
 6   2021.06  172 non-null    object
 7   2021.07  172 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


In [443]:
pm25_col = df_pm25.iloc[:,2:-1].columns.tolist()

In [444]:
df_pm25.drop(columns=pm25_col, inplace=True)

In [445]:
advanced_replace(df_pm25, '2021.07','-',r'[^0-9.0-9]')
df_pm25['2021.07'] = df_pm25['2021.07'].astype(float)

In [446]:
df_pm25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [447]:
df_pm25.describe()

,2021.07
count,172.000000
mean,11.558140
std,2.888188
min,0.000000
25%,10.000000
50%,11.000000
75%,13.000000
max,19.000000


In [448]:
mean = df_pm25.iloc[:,2].mean()
std = df_pm25.iloc[:,2].std()
min = df_pm25.iloc[:,2].min()
max = df_pm25.iloc[:,2].max()
show_norm(mean,std,min,max)

In [449]:
busan_pm25 = df_pm25[df_pm25['구분(2)']=='부산광역시'].loc[2,'2021.07']
pm25_pro = cal_norm(mean,std,min,max,busan_pm25)

In [450]:
pm10_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\미세먼지_PM10__월별_도시별_대기오염도.csv'
df_pm10 = pd.read_csv(pm10_file_path, encoding='cp949')
df_pm10

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,43,65,41,52,32,22
1,서울특별시,서울특별시,48,67,42,61,33,24
2,부산광역시,부산광역시,37,60,37,36,26,18
3,대구광역시,대구광역시,41,59,42,41,35,19
4,인천광역시,인천광역시,46,69,42,59,36,29
...,...,...,...,...,...,...,...,...
167,경상남도,합천,29,44,32,36,27,17**
168,경상남도,도평균,33,53,34,36,27,20
169,제주특별자치도,제주특별자치도,36,76,37,44,27,17
170,제주특별자치도,서귀포,33,68,35,40,23,21


In [451]:
pm10_col = df_pm10.iloc[:,2:-1].columns.tolist()

In [452]:
df_pm10.drop(columns=pm10_col, inplace=True)

In [453]:
advanced_replace(df_pm10, '2021.07', '-', r'[^0-9.0-9]')
df_pm10['2021.07'] = df_pm10['2021.07'].astype(float)

In [454]:
df_pm10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [455]:
df_pm10.describe()

,2021.07
count,172.000000
mean,20.750000
std,4.725352
min,0.000000
25%,18.000000
50%,20.000000
75%,23.000000
max,36.000000


In [456]:
mean = df_pm10.iloc[:,2].mean()
std = df_pm10.iloc[:,2].std()
min = df_pm10.iloc[:,2].min()
max = df_pm10.iloc[:,2].max()
show_norm(mean,std,min,max)

In [457]:
busan_pm10 = df_pm10[df_pm10['구분(2)']=='부산광역시'].loc[2,'2021.07']
pm10_pro = cal_norm(mean,std,min,max,busan_pm10)

## 상위 요소 - 사회적 요인

### 행정구역 인구 데이터 불러오기

In [458]:
people_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\행정구역_시군구_별__성별_인구수.csv'
df_people = pd.read_csv(people_file_path, encoding='cp949', header=1)
df_people

,행정구역(시군구)별,총인구수 (명)
0,전국,51592660
1,서울특별시,9500480
2,종로구,144005
3,중구,122087
4,용산구,222009
...,...,...
287,거창군,60756
288,합천군,42613
289,제주특별자치도,677413
290,제주시,493076


### 부산광역시 인구 데이터 추출

In [459]:

df_busan_people

,총인구수 (명)
행정구역(시군구)별,
강서구,143638.0
금정구,225033.0
기장군,176505.0
남구,260617.0
동구,88257.0
동래구,273976.0
부산진구,351022.0
북구,280852.0
사상구,206365.0


In [460]:
df_busan_people = df_people[28:44].sort_values('행정구역(시군구)별')
df_busan_people.set_index('행정구역(시군구)별',inplace=True)
df_busan_people['총인구수 (명)'] = df_busan_people['총인구수 (명)'].astype(float)
df_busan_people

,총인구수 (명)
행정구역(시군구)별,
강서구,143638.0
금정구,225033.0
기장군,176505.0
남구,260617.0
동구,88257.0
동래구,273976.0
부산진구,351022.0
북구,280852.0
사상구,206365.0


In [461]:
df_busan_people.describe()

,총인구수 (명)
count,16.000000
mean,208763.375000
std,98697.592227
min,40585.000000
25%,135080.250000
50%,206521.000000
75%,275695.000000
max,393130.000000


In [490]:
busan_people_pro = cal_norm(df_busan_people['총인구수 (명)'].mean(), df_busan_people['총인구수 (명)'].std(), df_busan_people['총인구수 (명)'].min(), 
df_busan_people['총인구수 (명)'].max(), df_busan_people.loc['사하구']['총인구수 (명)'])

### 유동인구 데이터 불러오기

In [463]:
foottraffic_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 유동인구 데이터.csv'
df_foottraffic = pd.read_csv(foottraffic_file_path, encoding='cp949')
df_foottraffic

,구군,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,월 평균
0,강서구,"325,284.39","323,049.87","335,120.06","340,570.52","332,289.77","340,678.45","337,488.64","328,374.74","328,764.38","333,414.26","340,405.10","337,753.15","333,599.44"
1,금정구,"482,598.89","477,047.11","484,669.59","490,864.46","487,435.45","484,936.00","483,898.58","482,964.23","487,082.06","484,633.90","485,502.59","488,392.99","485,002.15"
2,기장군,"341,181.17","343,398.79","342,489.52","345,184.20","348,793.68","348,500.32","350,763.40","349,328.59","348,077.97","348,449.77","345,905.89","344,245.29","346,359.88"
3,남구,"539,072.58","526,693.91","524,611.67","524,961.53","526,758.54","531,078.89","529,810.42","530,199.16","531,658.35","525,870.37","528,106.40","531,610.18","529,202.67"
4,동구,"185,466.33","185,459.30","189,082.61","192,702.25","189,819.14","191,618.73","189,864.16","189,368.74","190,328.67","191,182.03","194,732.77","193,341.86","190,247.22"
5,동래구,"553,655.76","549,718.60","543,091.96","535,439.53","533,908.61","528,124.07","527,730.64","525,468.79","532,656.70","527,498.20","520,126.56","524,788.88","533,517.36"
6,부산진구,"757,261.10","760,742.04","760,212.63","761,728.33","765,460.09","764,636.01","759,638.81","754,837.78","763,942.81","761,731.75","763,818.34","768,957.14","761,913.90"
7,북구,"559,133.62","554,946.17","543,084.57","543,479.25","543,145.91","540,381.96","542,289.98","546,905.11","551,394.74","542,887.88","539,732.97","545,952.52","546,111.22"
8,사상구,"496,102.34","492,871.78","493,054.71","496,932.63","494,094.59","494,207.66","492,973.61","490,296.85","493,842.47","491,808.21","493,815.86","495,297.44","493,774.85"
9,사하구,"641,541.35","632,045.91","628,461.73","633,805.98","630,333.83","629,635.14","630,260.69","631,377.03","633,402.47","627,551.97","625,380.25","631,475.21","631,272.63"


In [464]:
foottraffic_col = df_foottraffic.iloc[:,1:-1].columns.tolist()

In [465]:
df_foottraffic.drop(columns=foottraffic_col, inplace=True)
df_foottraffic.set_index('구군',inplace=True)
df_foottraffic

,월 평균
구군,
강서구,"333,599.44"
금정구,"485,002.15"
기장군,"346,359.88"
남구,"529,202.67"
동구,"190,247.22"
동래구,"533,517.36"
부산진구,"761,913.90"
북구,"546,111.22"
사상구,"493,774.85"


In [466]:
advanced_replace(df_foottraffic, '월 평균','-',r'[^0-9.0-9]')
df_foottraffic['월 평균'] = df_foottraffic['월 평균'].astype(float)

In [467]:
df_foottraffic.describe()

,월 평균
count,16.000000
mean,441344.991250
std,205145.538509
min,121290.850000
25%,311221.577500
50%,440973.920000
75%,536665.825000
max,867841.630000


In [468]:
# 부산 행정구 단위 유동인구 정규분포
show_norm(df_foottraffic['월 평균'].mean(),df_foottraffic['월 평균'].std(),df_foottraffic['월 평균'].min(),df_foottraffic['월 평균'].max())

In [469]:
# 사하구 유동인구 확률 밀도 계산 
busan_foottraffic = df_foottraffic.loc['사하구']['월 평균']
foottraffic_pro = cal_norm(df_foottraffic['월 평균'].mean(),df_foottraffic['월 평균'].std(),df_foottraffic['월 평균'].min(),df_foottraffic['월 평균'].max(),busan_foottraffic)
foottraffic_pro


0.8177299761126527

### 친환경 자동차 등록수 데이터 불러오기

In [470]:
vehicle_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\연료별 자동차 등록 수.csv'
df_vehicle = pd.read_csv(vehicle_file_path, encoding='cp949')
drop_col = df_vehicle.iloc[:,1:3].columns.tolist()
df_vehicle.drop(columns=drop_col,inplace=True)
df_vehicle

,연료,서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주,계
0,휘발유,"1,649,518","714,221","584,780","818,874","312,081","341,412","297,231","94,929","3,056,798","365,724","403,958","525,588","398,182","466,016","654,776","886,361","302,434","11,872,883"
1,경유,"1,077,183","550,411","449,206","627,029","266,237","247,724","217,553","66,126","2,450,214","371,277","380,402","516,001","430,676","547,999","664,042","736,135","253,205","9,851,420"
2,엘피지,"240,940","111,371","106,654","113,654","88,883","67,456","36,101","12,985","404,449","58,335","70,304","89,210","93,615","111,494","127,818","139,592","58,412","1,931,273"
3,전기,"45,133","16,059","18,076","15,191","6,233","9,550","3,948","2,180","50,111","9,117","10,421","12,142","8,931","10,658","13,018","14,957","26,931","272,656"
4,하이브리드(휘발유+전기),"147,022","61,167","48,320","90,275","24,091","24,167","19,593","11,034","256,643","24,001","25,842","32,669","25,448","38,154","36,999","56,219","30,057","951,701"
5,하이브리드(경유+전기),"2,656","4,187","2,146","7,672",464,393,194,141,"3,964",286,580,461,694,486,405,"1,774",341,"26,844"
6,하이브리드(LPG+전기),"1,601",735,806,696,628,481,457,153,"3,453",611,639,728,785,833,974,"1,034",335,"14,949"
7,기타연료,"7,172","16,097","4,545","13,915","4,036","3,830","5,205","1,208","29,809","4,376","6,918","9,574","5,182","9,666","10,974","9,161",886,"142,554"
8,수소,"2,435","1,473",352,"1,139","1,018","1,045","2,395",205,"4,004","1,959","1,095",908,"1,435",448,77,"1,899",5,"21,892"
9,총합,"3,173,660","1,475,721","1,214,885","1,688,445","703,671","696,058","582,677","188,961","6,259,445","835,686","900,159","1,187,281","964,948","1,185,754","1,509,083","1,847,132","672,606","25,086,172"


In [471]:
# 전국 자동차 수 통계값 구하기 (전기, 수소)
col = df_vehicle.iloc[:,1:].columns.tolist()
for c in col:
    advanced_replace(df_vehicle,c,'-',r'[^0-9.0-9]')
    df_vehicle[c] = df_vehicle[c].astype(int)
df_elec_vehicle = df_vehicle.iloc[3:7,:-1].sum()
df_elec_vehicle = df_elec_vehicle[1:]

df_elec_vehicle = df_elec_vehicle.astype(int)
df_elec_vehicle.describe()


count        17.000000
mean      74479.411765
std       75151.087588
min       13508.000000
25%       34591.000000
50%       50131.000000
75%       73984.000000
max      314171.000000
dtype: float64

In [472]:
df_hidro_vehicle = df_vehicle.iloc[8,1:-1]
df_hidro_vehicle = df_hidro_vehicle.astype(int)
df_hidro_vehicle.describe()

count      17.000000
mean     1287.764706
std      1030.779531
min         5.000000
25%       448.000000
50%      1095.000000
75%      1899.000000
max      4004.000000
Name: 8, dtype: float64

In [473]:
# 전국 전기차 정규분포
show_norm(df_elec_vehicle.mean(), df_elec_vehicle.std(), df_elec_vehicle.min(), df_elec_vehicle.max())
show_norm(df_hidro_vehicle.mean(), df_hidro_vehicle.std(), df_hidro_vehicle.min(), df_hidro_vehicle.max())

In [474]:
print(df_elec_vehicle)
print(df_hidro_vehicle)

서울    196412
부산     82148
대구     69348
인천    113834
광주     31416
대전     34591
울산     24192
세종     13508
경기    314171
강원     34015
충북     37482
충남     46000
전북     35858
전남     50131
경북     51396
경남     73984
제주     57664
dtype: int32
서울    2435
부산    1473
대구     352
인천    1139
광주    1018
대전    1045
울산    2395
세종     205
경기    4004
강원    1959
충북    1095
충남     908
전북    1435
전남     448
경북      77
경남    1899
제주       5
Name: 8, dtype: int32


In [475]:
elec_vehicle_pro = cal_norm(df_elec_vehicle.mean(), df_elec_vehicle.std(), df_elec_vehicle.min(), df_elec_vehicle.max(),df_elec_vehicle['부산'])
print(elec_vehicle_pro)
hidro_vehicle_pro = cal_norm(df_hidro_vehicle.mean(), df_hidro_vehicle.std(), df_hidro_vehicle.min(), df_hidro_vehicle.max(),df_hidro_vehicle['부산'])
print(hidro_vehicle_pro)

0.5056384417276619


0.5593075575000511


### LPG 충전소 현황 데이터 불러오기 (수소차)

In [511]:
LPG_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 LPG 충전소 현황(한국가스안전공사).csv'
df_LPG = pd.read_csv(LPG_file_path, encoding='cp949')
# 복합충전소를 위해 1500 m^2이상의 부지가 요구됨
df_LPG = df_LPG[df_LPG['면적'] >= 1500].reset_index(drop=True)
df_LPG

,행정구역,업소명,소재지,관리구분,위도,경도,면적,공시지가
0,부산 강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0
1,부산 강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0
2,부산 강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0
3,부산 강서구,르노삼성자동차(주),부산 강서구 르노삼성대로 61,자동차충전,35.095408,128.883215,1538.0,640100.0
4,부산 강서구,죽림LPG충전소,부산 강서구 호계로 102,자동차충전,35.200736,128.893462,1580.0,1017000.0
5,부산 강서구,가락충전소,부산 강서구 호계로 195,자동차충전,35.208847,128.890148,2600.0,1017000.0
6,부산 금정구,(주)해연,부산 금정구 개좌로 225-15,용기+탱크+13kg용기,35.230796,129.122388,3346.0,1370000.0
7,부산 금정구,(주)신진가스,부산 금정구 반송로 433,자동차충전,35.215006,129.111143,1646.4,2797000.0
8,부산 기장군,마라톤가스(주)청강충전소,부산 기장군 기장읍 기장대로 343 A동,자동차충전,35.225657,129.213988,2592.0,1069000.0
9,부산 기장군,한국에너지산업(주),부산 기장군 기장읍 기장대로413번길 38,용기+자동차+탱크+13kg용기,35.235543,129.215629,2473.0,1063000.0


In [495]:
df_LPG.shape[0]

63

In [496]:
LPG_group = df_LPG.groupby('행정구역').count()
LPG_group


,업소명,소재지,관리구분,위도,경도,면적,공시지가
행정구역,,,,,,,
부산 강서구,9,9,9,9,9,9,9
부산 금정구,3,3,3,3,3,3,3
부산 기장군,13,13,13,13,13,13,13
부산 남구,1,1,1,1,1,1,1
부산 동구,6,6,6,6,6,6,6
부산 동래구,1,1,1,1,1,0,0
부산 북구,4,4,4,4,4,4,4
부산 사상구,12,12,12,12,12,11,11
부산 사하구,6,6,6,6,6,6,6


In [501]:
# 복합충전소를 위해 1500 m^2의 부지가 요구됨
available_LPG = df_LPG[df_LPG['면적'] > 1500].loc['부산 사하구']['업소명']
print(num_available_LPG)
LPG_pro = cal_norm(LPG_group['업소명'].mean(),LPG_group['업소명'].std(),LPG_group['업소명'].min(),LPG_group['업소명'].max(),LPG_group.loc['부산 사하구']['업소명'])

31


### 전기차 충전소 현황 데이터 불러오기

In [480]:
EVCS_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 전기차 충전소 정보.csv'
df_EVCS = pd.read_csv(EVCS_file_path, encoding='cp949')
df_EVCS

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,위도,경도
0,환경부(한국자동차환경협회),kbs 부산방송총국,1,DC차데모+AC3상+DC콤보,부산광역시,수영구,부산광역시 수영구 수영로 429,35.144806,129.107595
1,환경부(한국자동차환경협회),kbs 부산방송총국,11,DC콤보,부산광역시,수영구,부산광역시 수영구 수영로 429,35.144806,129.107595
2,환경부(한국자동차환경협회),kbs 부산방송총국,12,DC콤보,부산광역시,수영구,부산광역시 수영구 수영로 429,35.144806,129.107595
3,환경부(한국자동차환경협회),국립부산과학관,11,DC콤보,부산광역시,기장군,부산광역시 기장군 기장읍 석산리 293,35.204002,129.212987
4,환경부(한국자동차환경협회),국립부산과학관,12,DC콤보,부산광역시,기장군,부산광역시 기장군 기장읍 석산리 293,35.204002,129.212987
...,...,...,...,...,...,...,...,...,...
6291,한국전자금융,부산 플러스타워,1,AC완속,부산광역시,강서구,부산광역시 강서구 명지국제1로 56-1플러스타워(3598-2 플러스타워),35.093685,128.905372
6292,한국전자금융,부산 플러스타워,2,AC완속,부산광역시,강서구,부산광역시 강서구 명지국제1로 56-1플러스타워(3598-2 플러스타워),35.093685,128.905372
6293,한국전자금융,부산 휴마트괴정점,1,AC완속,부산광역시,사하구,부산광역시 사하구 낙동대로 123휴마트괴정점(괴정동 358-1),35.103200,129.002528
6294,한국전자금융,부산 휴마트괴정점,2,AC완속,부산광역시,사하구,부산광역시 사하구 낙동대로 123휴마트괴정점(괴정동 358-1),35.103200,129.002528


In [481]:
# 부산 전체 충전기 수 구하기
df_EVCS.shape[0]

6296

In [482]:
EVCS_group = df_EVCS.groupby('시군구').count()
EVCS_group

,운영기관,충전소,충전기ID,충전기타입,지역,주소,위도,경도
시군구,,,,,,,,
강서구,465,465,465,465,465,465,465,465
금정구,311,311,311,311,311,311,311,311
기장군,504,504,504,504,504,504,504,504
남구,681,681,681,681,681,681,681,681
동구,119,119,119,119,119,119,119,119
동래구,471,471,471,471,471,471,471,471
부산진구,583,583,583,583,583,583,583,583
북구,353,353,353,353,353,353,353,353
사상구,315,315,315,315,315,315,315,315


In [483]:
EVCS_group['충전소'] = EVCS_group['충전소'].astype(int)
EVCS_pro = cal_norm(EVCS_group['충전소'].mean(),EVCS_group['충전소'].std(),EVCS_group['충전소'].min(),EVCS_group['충전소'].max(),EVCS_group.loc['사하구 ']['충전소'])

### 수소충전소 현황 데이터 불러오기

In [484]:
HVCS_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\수소충전소(환경부).csv'
df_HVCS = pd.read_csv(HVCS_file_path, encoding='cp949')
df_HVCS

,충전소,충전기,주소,이용가능시간
0,상암수소충전소,1,서울특별시 마포구 하늘공원로 108,8~20시(연중무휴)
1,마곡에코수소충전소,1,서울특별시 강서구 양천로 225,8~20시(일요일 휴무)
2,오곡수소충전소,1,서울특별시 강서구 벌말로 271,8~22시(연중무휴)
3,H국회수소충전소,1,서울특별시 영등포구 의사당대로 1,"평일 8~24시 / 주말, 공휴일 8~22시"
4,양재수소충전소,1,서울특별시 서초구 바우뫼로12길 73,8~22시(연중무휴)
...,...,...,...,...
103,하이넷 통영용남수소충전소,1,경상남도 통영시 용남면 남해안대로 38,"7~21시(매주 수요일 / 둘, 넷째주 일요일 휴무)"
104,김해수소충전소,2,경상남도 김해시 김해대로2596번길 53,"평일 8~21시 / 주말, 공휴일 9~18시"
105,하이넷 거제장평수소충전소,1,경상남도 거제시 거제대로 5050,6~24시(일요일 휴무)
106,양산시증산수소충전소,1,경상남도 양산시 물금읍 메기로 114,7~23시(연중무휴)


In [485]:
address_city = []
address_gu = []
for i in range(df_HVCS.shape[0]):
    strings = df_HVCS.iloc[i]['주소'].split()
    address_city.append(strings.pop(0))
    address_gu.append(strings.pop(0))
df_HVCS.insert(2,"시",address_city)
df_HVCS.insert(3,"구",address_gu)    
df_HVCS

,충전소,충전기,시,구,주소,이용가능시간
0,상암수소충전소,1,서울특별시,마포구,서울특별시 마포구 하늘공원로 108,8~20시(연중무휴)
1,마곡에코수소충전소,1,서울특별시,강서구,서울특별시 강서구 양천로 225,8~20시(일요일 휴무)
2,오곡수소충전소,1,서울특별시,강서구,서울특별시 강서구 벌말로 271,8~22시(연중무휴)
3,H국회수소충전소,1,서울특별시,영등포구,서울특별시 영등포구 의사당대로 1,"평일 8~24시 / 주말, 공휴일 8~22시"
4,양재수소충전소,1,서울특별시,서초구,서울특별시 서초구 바우뫼로12길 73,8~22시(연중무휴)
...,...,...,...,...,...,...
103,하이넷 통영용남수소충전소,1,경상남도,통영시,경상남도 통영시 용남면 남해안대로 38,"7~21시(매주 수요일 / 둘, 넷째주 일요일 휴무)"
104,김해수소충전소,2,경상남도,김해시,경상남도 김해시 김해대로2596번길 53,"평일 8~21시 / 주말, 공휴일 9~18시"
105,하이넷 거제장평수소충전소,1,경상남도,거제시,경상남도 거제시 거제대로 5050,6~24시(일요일 휴무)
106,양산시증산수소충전소,1,경상남도,양산시,경상남도 양산시 물금읍 메기로 114,7~23시(연중무휴)


In [486]:
df_HVCS_group = df_HVCS.groupby('시').count()
df_HVCS_group

,충전소,충전기,구,주소,이용가능시간
시,,,,,
강원도,7,7,7,7,7
경기도,19,19,19,19,19
경상남도,12,12,12,12,12
경상북도,2,2,2,2,2
광주광역시,6,6,6,6,6
대구광역시,4,4,4,4,4
대전광역시,6,6,6,6,6
부산광역시,3,3,3,3,3
서울특별시,6,6,6,6,6


In [487]:
df_busan_HVCS = df_HVCS[df_HVCS['시']=='부산광역시'].reset_index(drop=True)
df_busan_HVCS

,충전소,충전기,시,구,주소,이용가능시간
0,서부산NK수소충전소,1,부산광역시,강서구,부산광역시 강서구 가락대로 347,"8~18시(토, 일요일 휴무)"
1,H부산수소충전소,3,부산광역시,사상구,부산광역시 사상구 학감대로 167,버스 5~22시 / 넥쏘 10~22시(연중무휴)
2,하이넷부산정관수소충,1,부산광역시,기장군,부산광역시 기장군 정관읍 신정관로 187,8~20시(월요일 휴무)


In [488]:
df_HVCS_group.describe()

,충전소,충전기,구,주소,이용가능시간
count,16.000000,16.000000,16.000000,16.000000,16.000000
mean,6.750000,6.750000,6.750000,6.750000,6.750000
std,4.374167,4.374167,4.374167,4.374167,4.374167
min,2.000000,2.000000,2.000000,2.000000,2.000000
25%,3.750000,3.750000,3.750000,3.750000,3.750000
50%,6.000000,6.000000,6.000000,6.000000,6.000000
75%,9.000000,9.000000,9.000000,9.000000,9.000000
max,19.000000,19.000000,19.000000,19.000000,19.000000


In [492]:
HVCS_pro = cal_norm(df_HVCS_group['충전소'].mean(),df_HVCS_group['충전소'].std(),df_HVCS_group['충전소'].min(),df_HVCS_group['충전소'].max(),df_HVCS_group.loc['부산광역시']['충전소'])

### 교통 편의성 데이터

### 주변 충전소와의 거리 데이터

### 고속도로 여부

## 상위 요소 - 경제적 요소

### 전기차 충전기 정보 불러오기

### 충전소 설치 비용 불러오기 (전기차)
충전기 업체 마다 비용이 상이함
충전기 종류도 매우 많고 데이터 또한 없음

충전 사업 정부지원
- 1기 200만원, 2~5기 180만원, 6기 이상 150만원

총비용 
- 전기차 충전기 제품 비용 + 한전시설부담금

1. 파워큐브 - 비공용: 이동형 610,000원, 고정형 1,300,000원 / 
            - 공용: 주차면100면 이상 0원
2. kt - 충전기 가격 X
3. 지엔텔 - 충전기 가격 X
4. 에버온 - 충전기 가격 X
5. 삼성 - 비공용: 550,000원, 
        - 공용: (완속) 1,000,000원/1,600,000(2채널)  /  (급속) 58,000,000원(100kw), 40,000,000원(200kw)  
               

### 충전소 설치 비용 데이터 불러오기 (수소차) 
250kg급 수소충전소 구축비용을 종류별로 따져보면 파이프라인 방식 24억원, 튜브트레일러방식 22억원, 이동식 18억원이고, 현장공급방식중 천연가스개질방식은 46억원, 수전해방식은 31억원 가량 소요된다. 천연가스 개질방식이 가장 비싸고, 이동식이 가장 저렴

### 총 유지비용

### 주차 구획 수 데이터 불러오기

In [ ]:
parking_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산광역시 공영주차장 정보 조회.csv'
df_parking = pd.read_csv(parking_file_path, encoding='cp949')
df_parking

,운영형태,관리기관명,주차장명,주차장관리번호,소재지지번주소,소재지도로명주소,전화번호,주차장유형,주차구획수,평일운영시작시각,...,부제시행구분,운영요일,요금정보,주차기본시간,추가단위시간,추가단위요금,1일주차권요금적용시간,결제방법,특기사항,실시간주차면수
0,구(군)직영,부산광역시 사상구청,감전동주변,2010000013,-,사상구 감전동 589-2,051-310-4501,노외,26,0:00,...,-,-,-,10,10,0,-,-,-,-
1,구(군)직영,부산광역시 사상구청,장성주차장,2010000015,-,사상구 주례2동 92-278,051-310-4501,노외,16,0:00,...,-,-,-,10,10,0,-,-,-,-
2,민간위탁,부산광역시 사상구청,엄궁초등학교지하주차장,2010000016,사상구 엄궁로191번길 23 (엄궁동 47),-,051-310-4503,노외,93,0:00,...,-,-,-,10,10,100,-,-,-,-
3,구(군)직영,부산광역시 사상구청,일신주택주변,2011000003,-,사상구 주례3동 545-3,051-310-4501,노외,10,0:00,...,-,-,-,10,10,0,-,-,-,-
4,구(군)직영,부산광역시 사상구청,주례중학교주변,2012000004,-,사상구 주례2동 82-13,051-310-4501,노외,8,0:00,...,-,-,-,10,10,0,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,구(군)직영,부산광역시 기장군청,정관읍 상업지역 노상주차장,2016000017,-,기장군 정관면 732,-,노상,46,0:00,...,-,-,-,10,10,0,-,-,-,-
611,민간운영,부산광역시 기장군청,1구역 2소,2008000002,-,기장군 기장읍 280-2,051-792-4719,노상,12,9:00,...,-,-,-,10,10,200,-,-,-,-
612,민간운영,부산광역시 기장군청,1구역 3소,2008000003,-,기장군 기장읍 444-1,051-792-4719,노상,13,9:00,...,-,-,-,10,10,200,-,-,-,-
613,민간운영,부산광역시 기장군청,3구역 1소,2008000006,-,기장군 기장읍 418-12,051-792-4719,노상,24,9:00,...,-,-,-,10,10,200,-,-,-,-


### 주차 구획수 통계 구하기

In [ ]:
df_parking_area = df_parking['주차구획수'].astype(int)
df_parking_area.describe()

count    615.000000
mean      62.378862
std       70.372193
min       -4.000000
25%       18.000000
50%       38.000000
75%       79.000000
max      414.000000
Name: 주차구획수, dtype: float64

In [ ]:
show_norm(df_parking_area.mean(),df_parking_area.std(),df_parking_area.min(),df_parking_area.max())
parking_area_pro = cal_norm(df_parking_area.mean(),df_parking_area.std(),df_parking_area.min(),df_parking_area.max(),df_parking_area.iloc[610])
parking_area_pro

0.381979238207332

## 상위 요소 - 기술적 요소

### 복합 충전소 여부 (수소차)

기존의 LPG 충전소 부지에 수소충전소를 세우는 경우 "적합" 100%로 설정, 반대의 경우 "부적합" 100%로 설정함

### 충전시간에 따른 충전방식 (전기차, 정성적) 
 유동인구가 해당지역의 거주인구보다 많은 지역에 해당하는 경우 급속충전의 이용율이 높음
 
 따라서 "급속충전" 100%로 설정, 반대의 경우는 "완속충전" 100%로 설정함  

In [ ]:
# 초기 충전기 타입의 확률는 0.5  
charger_pro = 0.5

# 유동인구와 고정인구에 따른 충전방식 적합도 계산 
def charge_type(foottraffic_pro,busan_people_pro,charger_pro):
    '''
    foottraffic_pro : 유동인구 적합도
    busan_people_pro : 고정인구 적합도
    charger_pro : 설정하고자 하는 충전기 타입의 적합도 비율 초기값(0~1)
    '''
    
    if foottraffic_pro >= busan_people_pro:
        charger_type = 'Fast Charge'
    else:
        charger_type = 'Standard Charge'

    # 유동인구와 기존인구와 차이가 나는 만큼 더 많은 확률 부여
    if charger_type == 'Fast Charge':
        diff = foottraffic_pro - busan_people_pro
        charger_pro += diff
        if charger_pro > 1:
            charger_pro = 1.0
        print('Fast Charge:', charger_pro)
        print('Standard Charge:', 1-charger_pro)
    elif charger_type == 'Standard Charge':
        diff = busan_people_pro - foottraffic_pro
        charger_pro += diff
        if charger_pro > 1:
            charger_pro = 1.0
        print('Fast Charge:', 1-charger_pro)
        print('Standard Charge:', charger_pro)
    return charger_pro

Fast Charge: 0.48410058719061144
Standard Charge: 0.5158994128093886


### 결합 확률로 상위 요인 도출하기


In [493]:
variable_enviroment_E = []
variable_enviroment_H = []
variable_social_E = []
variable_social_H = []
variable_economic_E = []
variable_economic_H = []
variable_technical_E = []
variable_technical_H = []

weight_enviroment = [1/6,1/6,1/6,1/6,1/6,1/6] # co, so2, no2, ozone, pm10, pm25 
weight_social_E = [] # 고정인구, 유동인구, 전기차 수, 전기차 충전소 수
weight_social_H = [] # 고정인구, 유동인구, 수소차 수, 수소차 충전소 수
weight_economic_E = [] # 주차 구획수
weight_economic_H = [] #
weight_technical_E = [] # 급속/완속 적합성
weight_technical_H = [] # LPG 충전소 수 

variable_enviroment_E.append(co_pro)
variable_enviroment_E.append(so2_pro)
variable_enviroment_E.append(no2_pro)
variable_enviroment_E.append(ozone_pro)
variable_enviroment_E.append(pm10_pro)
variable_enviroment_E.append(pm25_pro)

# 환경변수는 전기차와 수소차 동일
variable_enviroment_H = variable_enviroment_E

variable_social_E.append(busan_people_pro)
variable_social_E.append(foottraffic_pro)
variable_social_E.append(elec_vehicle_pro)
variable_social_E.append(1-EVCS_pro)

variable_social_H.append(busan_people_pro)
variable_social_H.append(foottraffic_pro)
variable_social_H.append(hidro_vehicle_pro)
variable_social_H.append(1-HVCS_pro)

variable_economic_E.append(parking_area_pro)


variable_technical_E.append(charger_pro)

variable_technical_H.append(LPG_pro)

print("환경변수(전기,수소)",variable_enviroment_E)
print("사회변수(전기)",variable_social_E)
print("사회변수(수소)",variable_social_H)
print("경제변수(전기)",variable_economic_E)
print("경제변수(수소)",variable_economic_H)
print("기술변수(전기)",variable_technical_E)
print("기술변수(수소)",variable_technical_H)

환경변수(전기,수소) [0.064919397312361, 0.6640367362231872, 0.7579343076402061, 0.23346014669602194, 0.2802943598724571, 0.18788293015230972]
사회변수(전기) [0.8336293889220413, 0.8177299761126527, 0.5056384417276619, 0.5764770849672374]
사회변수(수소) [0.8336293889220413, 0.8177299761126527, 0.5593075575000511, 0.8223621499985667]
경제변수(전기) [0.381979238207332]
경제변수(수소) []
기술변수(전기) [0.5158994128093886]
기술변수(수소) [0.60763445399739]


In [ ]:
# 결합 확률 구하기
def combine_pro(list, weight):
    result = 0.0
    for i in range(len(list)):
        pro = list.pop(0)
        wei = weight.pop(0)
        
        result += pro*wei
    return result